# Hash Table (Bảng Băm) trong Python

## 1. Khái niệm về Hash Table

**Hash Table** (Bảng băm) là một cấu trúc dữ liệu cho phép lưu trữ và truy xuất dữ liệu một cách rất hiệu quả. Hash Table sử dụng kỹ thuật **hashing** để ánh xạ khóa (key) thành chỉ số trong một mảng.

### Đặc điểm chính:

- **Truy xuất nhanh**: Thời gian truy xuất trung bình O(1) - constant time
- **Lưu trữ cặp key-value**: Mỗi phần tử là một cặp khóa-giá trị
- **Sử dụng hàm băm (Hash Function)**: Chuyển đổi key thành chỉ số trong mảng
- **Xử lý đụng độ (Collision Handling)**: Giải quyết khi nhiều key có cùng hash value

### Tại sao sử dụng Hash Table?

- **Tìm kiếm cực nhanh**: O(1) trung bình so với O(n) của list
- **Thêm/xóa nhanh**: O(1) trung bình
- **Sử dụng trong Python dict**: Dictionary trong Python được triển khai bằng hash table
- **Ứng dụng rộng rãi**: Caching, database indexing, compiler symbol tables

## 2. Các thành phần của Hash Table

### 2.1. Hash Function (Hàm băm)

**Hash Function** là hàm chuyển đổi một key thành một giá trị số nguyên (hash value), sau đó ánh xạ giá trị này thành chỉ số trong mảng.

**Yêu cầu của Hash Function tốt:**
- **Deterministic**: Cùng một key luôn cho cùng một hash value
- **Phân bố đều**: Các key khác nhau nên có hash value khác nhau (càng nhiều càng tốt)
- **Tính toán nhanh**: O(1) thời gian

### 2.2. Hash Collision (Đụng độ)

**Hash Collision** xảy ra khi hai key khác nhau có cùng hash value. Có 2 cách xử lý phổ biến:

1. **Chaining (Liên kết)**: Lưu nhiều cặp key-value trong cùng một bucket (thường dùng linked list)
2. **Open Addressing (Địa chỉ mở)**: Tìm vị trí trống tiếp theo trong mảng

### 2.3. Các phương thức chính

- `hash(key)`: Tính hash value của key
- `insert(key, value)`: Thêm cặp key-value vào hash table
- `get(key)`: Lấy value theo key
- `delete(key)`: Xóa cặp key-value
- `contains(key)`: Kiểm tra key có tồn tại không

## 3. Triển khai Hash Table - Phần 1: Hash Function cơ bản

Bắt đầu với việc hiểu hash function đơn giản nhất:

In [ ]:
# Hash Function đơn giản cho số nguyên
def simple_hash_integer(key, size):
    """
    Hash function đơn giản cho số nguyên
    Sử dụng phép chia lấy dư (modulo)
    """
    return key % size

# Ví dụ
size = 10
keys = [5, 15, 25, 35, 45]
print("Hash values cho các keys:")
for key in keys:
    hash_value = simple_hash_integer(key, size)
    print(f"Key {key:3d} -> Hash: {hash_value}")

In [ ]:
# Hash Function cho chuỗi (string)
def simple_hash_string(key, size):
    """
    Hash function cho chuỗi
    Tính tổng ASCII của các ký tự rồi chia lấy dư
    """
    hash_value = 0
    for char in key:
        hash_value += ord(char)  # ord() trả về mã ASCII của ký tự
    return hash_value % size

# Ví dụ
size = 10
strings = ["apple", "banana", "cherry", "date"]
print("Hash values cho các chuỗi:")
for s in strings:
    hash_value = simple_hash_string(s, size)
    print(f"'{s}' -> Hash: {hash_value}")

# Lưu ý: Có thể xảy ra collision!
print("\nKiểm tra collision:")
s1, s2 = "ab", "ba"
print(f"'{s1}' -> Hash: {simple_hash_string(s1, size)}")
print(f"'{s2}' -> Hash: {simple_hash_string(s2, size)}")
if simple_hash_string(s1, size) == simple_hash_string(s2, size):
    print("⚠️ COLLISION xảy ra!")

## 4. Triển khai Hash Table - Phần 2: Hash Function cải tiến

Hash function tốt hơn cho chuỗi sử dụng **polynomial rolling hash**:

In [ ]:
# Hash Function cải tiến cho chuỗi (polynomial rolling hash)
def improved_hash_string(key, size):
    """
    Sử dụng polynomial rolling hash
    Công thức: hash = (char_0 * p^0 + char_1 * p^1 + ...) % m
    
    p: số nguyên tố (thường là 31 hoặc 37)
    m: kích thước hash table
    """
    p = 31  # Số nguyên tố
    hash_value = 0
    power = 1
    
    for char in key:
        hash_value = (hash_value + ord(char) * power) % size
        power = (power * p) % size
    
    return hash_value

# So sánh với hash function đơn giản
size = 10
strings = ["apple", "banana", "cherry", "ab", "ba"]

print("So sánh hash functions:")
print(f"{'String':<10} {'Simple Hash':<12} {'Improved Hash':<15}")
print("-" * 40)
for s in strings:
    simple = simple_hash_string(s, size)
    improved = improved_hash_string(s, size)
    print(f"'{s}':{len(s)<6} {simple:<12} {improved:<15}")

## 5. Triển khai Hash Table - Phần 3: Hash Table cơ bản (không xử lý collision)

Bắt đầu với hash table đơn giản, chưa xử lý collision:

In [ ]:
class SimpleHashTable:
    """
    Hash Table đơn giản - chưa xử lý collision
    Nếu có 2 key cùng hash value, value sau sẽ ghi đè value trước
    """
    
    def __init__(self, size=10):
        """Khởi tạo hash table với kích thước mặc định là 10"""
        self.size = size
        self.table = [None] * size  # Khởi tạo mảng với None
    
    def hash(self, key):
        """Hash function cho key (hỗ trợ cả int và string)"""
        if isinstance(key, int):
            return key % self.size
        elif isinstance(key, str):
            # Sử dụng polynomial rolling hash
            p = 31
            hash_value = 0
            power = 1
            for char in key:
                hash_value = (hash_value + ord(char) * power) % self.size
                power = (power * p) % self.size
            return hash_value
        else:
            # Với các kiểu khác, chuyển sang string rồi hash
            return self.hash(str(key))
    
    def insert(self, key, value):
        """Thêm cặp key-value vào hash table"""
        index = self.hash(key)
        self.table[index] = (key, value)  # Lưu tuple (key, value)
        print(f"✓ Đã thêm '{key}' -> {value} tại index {index}")
    
    def get(self, key):
        """Lấy value theo key"""
        index = self.hash(key)
        if self.table[index] is None:
            return None
        stored_key, value = self.table[index]
        if stored_key == key:
            return value
        return None  # Key không khớp (có thể do collision)
    
    def delete(self, key):
        """Xóa cặp key-value"""
        index = self.hash(key)
        if self.table[index] is not None:
            stored_key, _ = self.table[index]
            if stored_key == key:
                self.table[index] = None
                print(f"✓ Đã xóa '{key}' tại index {index}")
                return True
        return False
    
    def contains(self, key):
        """Kiểm tra key có tồn tại không"""
        return self.get(key) is not None
    
    def display(self):
        """Hiển thị toàn bộ hash table"""
        print("\n=== Hash Table ===")
        for i in range(self.size):
            if self.table[i] is not None:
                key, value = self.table[i]
                print(f"Index {i}: '{key}' -> {value}")
            else:
                print(f"Index {i}: None")
        print("=" * 30)

In [ ]:
# Kiểm tra SimpleHashTable
ht = SimpleHashTable(size=5)

# Thêm các phần tử
ht.insert("apple", 100)
ht.insert("banana", 200)
ht.insert("cherry", 300)
ht.insert(1, "one")
ht.insert(6, "six")  # 6 % 5 = 1, sẽ collision với key=1!

ht.display()

# Truy xuất
print(f"\nget('apple'): {ht.get('apple')}")
print(f"get('banana'): {ht.get('banana')}")
print(f"get(1): {ht.get(1)}")
print(f"get(6): {ht.get(6)}")  # Sẽ bị ghi đè hoặc None

# Kiểm tra collision
print(f"\ncontains('apple'): {ht.contains('apple')}")
print(f"contains('grape'): {ht.contains('grape')}")

# Xóa
ht.delete("banana")
ht.display()

## 6. Triển khai Hash Table - Phần 4: Xử lý Collision bằng Chaining

**Chaining** là kỹ thuật xử lý collision bằng cách lưu nhiều cặp key-value trong cùng một bucket (sử dụng list hoặc linked list).

In [ ]:
class HashTableWithChaining:
    """
    Hash Table với xử lý collision bằng Chaining
    Mỗi bucket là một list chứa các tuple (key, value)
    """
    
    def __init__(self, size=10):
        """Khởi tạo hash table với kích thước mặc định là 10"""
        self.size = size
        self.table = [[] for _ in range(size)]  # Mỗi bucket là một list rỗng
    
    def hash(self, key):
        """Hash function cho key"""
        if isinstance(key, int):
            return key % self.size
        elif isinstance(key, str):
            p = 31
            hash_value = 0
            power = 1
            for char in key:
                hash_value = (hash_value + ord(char) * power) % self.size
                power = (power * p) % self.size
            return hash_value
        else:
            return self.hash(str(key))
    
    def insert(self, key, value):
        """Thêm cặp key-value vào hash table (xử lý collision)"""
        index = self.hash(key)
        bucket = self.table[index]
        
        # Kiểm tra xem key đã tồn tại chưa
        for i, (k, v) in enumerate(bucket):
            if k == key:
                # Key đã tồn tại, cập nhật value
                bucket[i] = (key, value)
                print(f"✓ Đã cập nhật '{key}' -> {value} tại index {index}")
                return
        
        # Key chưa tồn tại, thêm mới
        bucket.append((key, value))
        print(f"✓ Đã thêm '{key}' -> {value} tại index {index} (bucket size: {len(bucket)})")
    
    def get(self, key):
        """Lấy value theo key"""
        index = self.hash(key)
        bucket = self.table[index]
        
        for k, v in bucket:
            if k == key:
                return v
        return None  # Không tìm thấy
    
    def delete(self, key):
        """Xóa cặp key-value"""
        index = self.hash(key)
        bucket = self.table[index]
        
        for i, (k, v) in enumerate(bucket):
            if k == key:
                bucket.pop(i)
                print(f"✓ Đã xóa '{key}' tại index {index}")
                return True
        return False
    
    def contains(self, key):
        """Kiểm tra key có tồn tại không"""
        return self.get(key) is not None
    
    def display(self):
        """Hiển thị toàn bộ hash table"""
        print("\n=== Hash Table (Chaining) ===")
        for i in range(self.size):
            bucket = self.table[i]
            if bucket:
                items = [f"'{k}':{v}" for k, v in bucket]
                print(f"Index {i}: [{', '.join(items)}]")
            else:
                print(f"Index {i}: []")
        print("=" * 40)

In [ ]:
# Kiểm tra HashTableWithChaining
ht = HashTableWithChaining(size=5)

# Thêm các phần tử - bao gồm collision
ht.insert("apple", 100)
ht.insert("banana", 200)
ht.insert("cherry", 300)
ht.insert(1, "one")
ht.insert(6, "six")   # 6 % 5 = 1 -> collision với key=1, nhưng vẫn OK!
ht.insert(11, "eleven")  # 11 % 5 = 1 -> collision nữa!

ht.display()

# Truy xuất
print(f"\nget('apple'): {ht.get('apple')}")
print(f"get(1): {ht.get(1)}")
print(f"get(6): {ht.get(6)}")  # ✅ Không còn bị ghi đè!
print(f"get(11): {ht.get(11)}")

# Kiểm tra
print(f"\ncontains('apple'): {ht.contains('apple')}")
print(f"contains('grape'): {ht.contains('grape')}")

# Xóa
ht.delete("banana")
ht.display()

## 7. Triển khai Hash Table - Phần 5: Xử lý Collision bằng Linear Probing

**Linear Probing** là kỹ thuật xử lý collision bằng cách tìm vị trí trống tiếp theo trong mảng khi collision xảy ra.

In [ ]:
class HashTableWithLinearProbing:
    """
    Hash Table với xử lý collision bằng Linear Probing
    Khi collision, tìm vị trí trống tiếp theo
    """
    
    def __init__(self, size=10):
        """Khởi tạo hash table"""
        self.size = size
        self.table = [None] * size
        self.count = 0  # Số phần tử hiện tại
    
    def hash(self, key):
        """Hash function cho key"""
        if isinstance(key, int):
            return key % self.size
        elif isinstance(key, str):
            p = 31
            hash_value = 0
            power = 1
            for char in key:
                hash_value = (hash_value + ord(char) * power) % self.size
                power = (power * p) % self.size
            return hash_value
        else:
            return self.hash(str(key))
    
    def _find_index(self, key):
        """
        Tìm index để insert hoặc get key
        Trả về index nếu tìm thấy vị trí phù hợp, None nếu không
        """
        index = self.hash(key)
        start_index = index
        
        # Linear probing: tìm vị trí trống hoặc có cùng key
        while True:
            if self.table[index] is None:
                return index  # Vị trí trống
            stored_key, _ = self.table[index]
            if stored_key == key:
                return index  # Tìm thấy key
            index = (index + 1) % self.size  # Vị trí tiếp theo (wrap around)
            if index == start_index:
                return None  # Hash table đầy
    
    def _find_key_index(self, key):
        """Tìm index của key (chỉ dùng cho get/delete)"""
        index = self.hash(key)
        start_index = index
        
        while self.table[index] is not None:
            stored_key, _ = self.table[index]
            if stored_key == key:
                return index
            index = (index + 1) % self.size
            if index == start_index:
                break
        return None
    
    def insert(self, key, value):
        """Thêm cặp key-value (sử dụng linear probing khi collision)"""
        if self.count >= self.size:
            print("⚠️ Hash table đầy!")
            return False
        
        index = self._find_index(key)
        if index is None:
            print("⚠️ Không thể thêm - hash table đầy!")
            return False
        
        if self.table[index] is None:
            self.count += 1
            print(f"✓ Đã thêm '{key}' -> {value} tại index {index}")
        else:
            print(f"✓ Đã cập nhật '{key}' -> {value} tại index {index}")
        
        self.table[index] = (key, value)
        return True
    
    def get(self, key):
        """Lấy value theo key"""
        index = self._find_key_index(key)
        if index is not None:
            _, value = self.table[index]
            return value
        return None
    
    def delete(self, key):
        """Xóa cặp key-value"""
        index = self._find_key_index(key)
        if index is not None:
            self.table[index] = None
            self.count -= 1
            print(f"✓ Đã xóa '{key}' tại index {index}")
            
            # Rehash các phần tử tiếp theo trong cluster
            next_index = (index + 1) % self.size
            while self.table[next_index] is not None:
                k, v = self.table[next_index]
                self.table[next_index] = None
                self.count -= 1
                self.insert(k, v)  # Reinsert
                next_index = (next_index + 1) % self.size
            
            return True
        return False
    
    def contains(self, key):
        """Kiểm tra key có tồn tại không"""
        return self.get(key) is not None
    
    def display(self):
        """Hiển thị toàn bộ hash table"""
        print(f"\n=== Hash Table (Linear Probing) [Count: {self.count}/{self.size}] ===")
        for i in range(self.size):
            if self.table[i] is not None:
                key, value = self.table[i]
                print(f"Index {i}: '{key}' -> {value}")
            else:
                print(f"Index {i}: None")
        print("=" * 50)

In [ ]:
# Kiểm tra HashTableWithLinearProbing
ht = HashTableWithLinearProbing(size=5)

# Thêm các phần tử
ht.insert("apple", 100)
ht.insert("banana", 200)
ht.insert("cherry", 300)
ht.insert(1, "one")
ht.insert(6, "six")  # 6 % 5 = 1 -> collision, sẽ tìm vị trí tiếp theo

ht.display()

# Truy xuất
print(f"\nget('apple'): {ht.get('apple')}")
print(f"get(1): {ht.get(1)}")
print(f"get(6): {ht.get(6)}")  # ✅ Tìm được!

# Xóa
ht.delete("banana")
ht.display()

## 8. So sánh các kỹ thuật xử lý Collision

### Chaining vs Linear Probing

| Đặc điểm | Chaining | Linear Probing |
|----------|----------|----------------|
| **Lưu trữ** | Mỗi bucket là list | Mỗi vị trí là một entry |
| **Khi collision** | Thêm vào list | Tìm vị trí trống tiếp theo |
| **Load factor** | Có thể > 1 | Phải ≤ 1 |
| **Xóa** | Đơn giản | Cần rehash |
| **Tốc độ** | Tốt khi ít collision | Nhanh hơn khi ít collision |

**Load Factor** = Số phần tử / Kích thước hash table
- Load factor càng cao → collision càng nhiều → hiệu suất giảm
- Thường giữ load factor < 0.7

## 9. Sử dụng Hash Table trong Python

### 9.1. Dictionary trong Python là Hash Table

Python's `dict` được triển khai bằng hash table với chaining!

In [ ]:
# Dictionary trong Python sử dụng hash table
my_dict = {
    "apple": 100,
    "banana": 200,
    "cherry": 300
}

# Các thao tác cơ bản
print("Thêm phần tử:")
my_dict["date"] = 400
print(my_dict)

print("\nTruy xuất:")
print(f"apple: {my_dict['apple']}")
print(f"banana: {my_dict.get('banana')}")
print(f"grape: {my_dict.get('grape', 'Not found')}")

print("\nKiểm tra tồn tại:")
print(f"'apple' in my_dict: {'apple' in my_dict}")

print("\nXóa phần tử:")
del my_dict["banana"]
print(my_dict)

# Hiệu suất
import time

# Tạo dữ liệu lớn
large_dict = {i: i*2 for i in range(100000)}
large_list = list(range(100000))

# So sánh tốc độ tìm kiếm
target = 99999

# Hash table (dict) - O(1)
start = time.time()
_ = large_dict[target]
dict_time = time.time() - start

# List - O(n)
start = time.time()
_ = large_list.index(target)
list_time = time.time() - start

print(f"\nSo sánh tốc độ tìm kiếm (phần tử {target}):")
print(f"Hash Table (dict): {dict_time*1000:.4f} ms")
print(f"List: {list_time*1000:.4f} ms")
print(f"Hash Table nhanh hơn {list_time/dict_time:.1f} lần!")

## 10. Ứng dụng thực tế của Hash Table

### 10.1. Đếm tần suất xuất hiện

In [ ]:
# Đếm tần suất xuất hiện của các từ trong văn bản
def count_frequency(text):
    """Đếm số lần xuất hiện của mỗi từ"""
    words = text.lower().split()
    frequency = {}  # Hash table
    
    for word in words:
        frequency[word] = frequency.get(word, 0) + 1
    
    return frequency

text = "the quick brown fox jumps over the lazy dog the fox is quick"
freq = count_frequency(text)

print("Tần suất xuất hiện của các từ:")
for word, count in sorted(freq.items()):
    print(f"  '{word}': {count}")

### 10.2. Kiểm tra phần tử trùng lặp

In [ ]:
def has_duplicate(arr):
    """Kiểm tra mảng có phần tử trùng lặp không (sử dụng hash set)"""
    seen = set()  # Hash set (chỉ lưu keys, không lưu values)
    
    for num in arr:
        if num in seen:
            return True
        seen.add(num)
    return False

# Kiểm tra
arr1 = [1, 2, 3, 4, 5]
arr2 = [1, 2, 3, 2, 4]

print(f"{arr1} có trùng lặp? {has_duplicate(arr1)}")
print(f"{arr2} có trùng lặp? {has_duplicate(arr2)}")

### 10.3. Tìm cặp phần tử có tổng bằng target (Two Sum)

In [ ]:
def two_sum(nums, target):
    """
    Tìm 2 số trong mảng có tổng bằng target
    Trả về indices của 2 số đó
    Sử dụng hash table để đạt O(n) thay vì O(n²)
    """
    num_map = {}  # {num: index}
    
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_map:
            return [num_map[complement], i]
        num_map[num] = i
    
    return None

# Kiểm tra
nums = [2, 7, 11, 15]
target = 9
result = two_sum(nums, target)
print(f"Input: nums = {nums}, target = {target}")
if result:
    print(f"Output: {result} (vì nums[{result[0]}] + nums[{result[1]}] = {nums[result[0]]} + {nums[result[1]]} = {target})")
else:
    print("Không tìm thấy!")

## 11. Tổng kết

### Ưu điểm của Hash Table:
- ✅ **Truy xuất cực nhanh**: O(1) trung bình
- ✅ **Thêm/xóa nhanh**: O(1) trung bình
- ✅ **Linh hoạt**: Hỗ trợ nhiều kiểu key
- ✅ **Thực tế**: Dictionary trong Python sử dụng hash table

### Nhược điểm:
- ⚠️ **Collision**: Cần xử lý khi nhiều key có cùng hash
- ⚠️ **Không có thứ tự**: Không giữ thứ tự thêm vào (Python 3.7+ giữ thứ tự)
- ⚠️ **Bộ nhớ**: Cần nhiều bộ nhớ hơn array đơn giản

### Khi nào sử dụng:
- ✅ Cần tìm kiếm/thêm/xóa nhanh (O(1))
- ✅ Cần lưu trữ cặp key-value
- ✅ Không cần duyệt theo thứ tự
- ✅ Dữ liệu không quá lớn (memory constraint)

### Time Complexity:

| Thao tác | Best Case | Average Case | Worst Case |
|----------|-----------|--------------|------------|
| **Search** | O(1) | O(1) | O(n) |
| **Insert** | O(1) | O(1) | O(n) |
| **Delete** | O(1) | O(1) | O(n) |

*Worst case xảy ra khi tất cả keys có cùng hash value (rất hiếm với hash function tốt)*

## 12. Bài tập thực hành

Hãy thử tự triển khai các chức năng sau:

1. **Triển khai Hash Table hoàn chỉnh** với load factor và resize
2. **Đếm số lần xuất hiện** của mỗi ký tự trong chuỗi
3. **Tìm phần tử đầu tiên không lặp lại** trong mảng
4. **Nhóm các từ đồng âm** (anagrams) sử dụng hash table

Bắt đầu thực hành ngay!

## 5. Triển khai Hash Table - Phần 3: Hash Table cơ bản (không xử lý collision)

Bắt đầu với hash table đơn giản, chưa xử lý collision:

In [ ]:
class SimpleHashTable:
    """
    Hash Table đơn giản - chưa xử lý collision
    Nếu có 2 key cùng hash value, value sau sẽ ghi đè value trước
    """
    
    def __init__(self, size=10):
        """Khởi tạo hash table với kích thước mặc định là 10"""
        self.size = size
        self.table = [None] * size  # Khởi tạo mảng với None
    
    def hash(self, key):
        """Hash function cho key (hỗ trợ cả int và string)"""
        if isinstance(key, int):
            return key % self.size
        elif isinstance(key, str):
            # Sử dụng polynomial rolling hash
            p = 31
            hash_value = 0
            power = 1
            for char in key:
                hash_value = (hash_value + ord(char) * power) % self.size
                power = (power * p) % self.size
            return hash_value
        else:
            # Với các kiểu khác, chuyển sang string rồi hash
            return self.hash(str(key))
    
    def insert(self, key, value):
        """Thêm cặp key-value vào hash table"""
        index = self.hash(key)
        self.table[index] = (key, value)  # Lưu tuple (key, value)
        print(f"✓ Đã thêm '{key}' -> {value} tại index {index}")
    
    def get(self, key):
        """Lấy value theo key"""
        index = self.hash(key)
        if self.table[index] is None:
            return None
        stored_key, value = self.table[index]
        if stored_key == key:
            return value
        return None  # Key không khớp (có thể do collision)
    
    def delete(self, key):
        """Xóa cặp key-value"""
        index = self.hash(key)
        if self.table[index] is not None:
            stored_key, _ = self.table[index]
            if stored_key == key:
                self.table[index] = None
                print(f"✓ Đã xóa '{key}' tại index {index}")
                return True
        return False
    
    def contains(self, key):
        """Kiểm tra key có tồn tại không"""
        return self.get(key) is not None
    
    def display(self):
        """Hiển thị toàn bộ hash table"""
        print("\n=== Hash Table ===")
        for i in range(self.size):
            if self.table[i] is not None:
                key, value = self.table[i]
                print(f"Index {i}: '{key}' -> {value}")
            else:
                print(f"Index {i}: None")
        print("=" * 30)

In [ ]:
# Kiểm tra SimpleHashTable
ht = SimpleHashTable(size=5)

# Thêm các phần tử
ht.insert("apple", 100)
ht.insert("banana", 200)
ht.insert("cherry", 300)
ht.insert(1, "one")
ht.insert(6, "six")  # 6 % 5 = 1, sẽ collision với key=1!

ht.display()

# Truy xuất
print(f"\nget('apple'): {ht.get('apple')}")
print(f"get('banana'): {ht.get('banana')}")
print(f"get(1): {ht.get(1)}")
print(f"get(6): {ht.get(6)}")  # Sẽ bị ghi đè hoặc None

# Kiểm tra collision
print(f"\ncontains('apple'): {ht.contains('apple')}")
print(f"contains('grape'): {ht.contains('grape')}")

# Xóa
ht.delete("banana")
ht.display()

## 6. Triển khai Hash Table - Phần 4: Xử lý Collision bằng Chaining

**Chaining** là kỹ thuật xử lý collision bằng cách lưu nhiều cặp key-value trong cùng một bucket (sử dụng list hoặc linked list).

In [ ]:
class HashTableWithChaining:
    """
    Hash Table với xử lý collision bằng Chaining
    Mỗi bucket là một list chứa các tuple (key, value)
    """
    
    def __init__(self, size=10):
        """Khởi tạo hash table với kích thước mặc định là 10"""
        self.size = size
        self.table = [[] for _ in range(size)]  # Mỗi bucket là một list rỗng
    
    def hash(self, key):
        """Hash function cho key"""
        if isinstance(key, int):
            return key % self.size
        elif isinstance(key, str):
            p = 31
            hash_value = 0
            power = 1
            for char in key:
                hash_value = (hash_value + ord(char) * power) % self.size
                power = (power * p) % self.size
            return hash_value
        else:
            return self.hash(str(key))
    
    def insert(self, key, value):
        """Thêm cặp key-value vào hash table (xử lý collision)"""
        index = self.hash(key)
        bucket = self.table[index]
        
        # Kiểm tra xem key đã tồn tại chưa
        for i, (k, v) in enumerate(bucket):
            if k == key:
                # Key đã tồn tại, cập nhật value
                bucket[i] = (key, value)
                print(f"✓ Đã cập nhật '{key}' -> {value} tại index {index}")
                return
        
        # Key chưa tồn tại, thêm mới
        bucket.append((key, value))
        print(f"✓ Đã thêm '{key}' -> {value} tại index {index} (bucket size: {len(bucket)})")
    
    def get(self, key):
        """Lấy value theo key"""
        index = self.hash(key)
        bucket = self.table[index]
        
        for k, v in bucket:
            if k == key:
                return v
        return None  # Không tìm thấy
    
    def delete(self, key):
        """Xóa cặp key-value"""
        index = self.hash(key)
        bucket = self.table[index]
        
        for i, (k, v) in enumerate(bucket):
            if k == key:
                bucket.pop(i)
                print(f"✓ Đã xóa '{key}' tại index {index}")
                return True
        return False
    
    def contains(self, key):
        """Kiểm tra key có tồn tại không"""
        return self.get(key) is not None
    
    def display(self):
        """Hiển thị toàn bộ hash table"""
        print("\n=== Hash Table (Chaining) ===")
        for i in range(self.size):
            bucket = self.table[i]
            if bucket:
                items = [f"'{k}':{v}" for k, v in bucket]
                print(f"Index {i}: [{', '.join(items)}]")
            else:
                print(f"Index {i}: []")
        print("=" * 40)

In [ ]:
# Kiểm tra HashTableWithChaining
ht = HashTableWithChaining(size=5)

# Thêm các phần tử - bao gồm collision
ht.insert("apple", 100)
ht.insert("banana", 200)
ht.insert("cherry", 300)
ht.insert(1, "one")
ht.insert(6, "six")   # 6 % 5 = 1 -> collision với key=1, nhưng vẫn OK!
ht.insert(11, "eleven")  # 11 % 5 = 1 -> collision nữa!

ht.display()

# Truy xuất
print(f"\nget('apple'): {ht.get('apple')}")
print(f"get(1): {ht.get(1)}")
print(f"get(6): {ht.get(6)}")  # ✅ Không còn bị ghi đè!
print(f"get(11): {ht.get(11)}")

# Kiểm tra
print(f"\ncontains('apple'): {ht.contains('apple')}")
print(f"contains('grape'): {ht.contains('grape')}")

# Xóa
ht.delete("banana")
ht.display()

## 7. Triển khai Hash Table - Phần 5: Xử lý Collision bằng Linear Probing

**Linear Probing** là kỹ thuật xử lý collision bằng cách tìm vị trí trống tiếp theo trong mảng khi collision xảy ra.

In [ ]:
class HashTableWithLinearProbing:
    """
    Hash Table với xử lý collision bằng Linear Probing
    Khi collision, tìm vị trí trống tiếp theo
    """
    
    def __init__(self, size=10):
        """Khởi tạo hash table"""
        self.size = size
        self.table = [None] * size
        self.count = 0  # Số phần tử hiện tại
    
    def hash(self, key):
        """Hash function cho key"""
        if isinstance(key, int):
            return key % self.size
        elif isinstance(key, str):
            p = 31
            hash_value = 0
            power = 1
            for char in key:
                hash_value = (hash_value + ord(char) * power) % self.size
                power = (power * p) % self.size
            return hash_value
        else:
            return self.hash(str(key))
    
    def _find_index(self, key):
        """
        Tìm index để insert hoặc get key
        Trả về index nếu tìm thấy vị trí phù hợp, None nếu không
        """
        index = self.hash(key)
        start_index = index
        
        # Linear probing: tìm vị trí trống hoặc có cùng key
        while True:
            if self.table[index] is None:
                return index  # Vị trí trống
            stored_key, _ = self.table[index]
            if stored_key == key:
                return index  # Tìm thấy key
            index = (index + 1) % self.size  # Vị trí tiếp theo (wrap around)
            if index == start_index:
                return None  # Hash table đầy
    
    def _find_key_index(self, key):
        """Tìm index của key (chỉ dùng cho get/delete)"""
        index = self.hash(key)
        start_index = index
        
        while self.table[index] is not None:
            stored_key, _ = self.table[index]
            if stored_key == key:
                return index
            index = (index + 1) % self.size
            if index == start_index:
                break
        return None
    
    def insert(self, key, value):
        """Thêm cặp key-value (sử dụng linear probing khi collision)"""
        if self.count >= self.size:
            print("⚠️ Hash table đầy!")
            return False
        
        index = self._find_index(key)
        if index is None:
            print("⚠️ Không thể thêm - hash table đầy!")
            return False
        
        if self.table[index] is None:
            self.count += 1
            print(f"✓ Đã thêm '{key}' -> {value} tại index {index}")
        else:
            print(f"✓ Đã cập nhật '{key}' -> {value} tại index {index}")
        
        self.table[index] = (key, value)
        return True
    
    def get(self, key):
        """Lấy value theo key"""
        index = self._find_key_index(key)
        if index is not None:
            _, value = self.table[index]
            return value
        return None
    
    def delete(self, key):
        """Xóa cặp key-value"""
        index = self._find_key_index(key)
        if index is not None:
            self.table[index] = None
            self.count -= 1
            print(f"✓ Đã xóa '{key}' tại index {index}")
            
            # Rehash các phần tử tiếp theo trong cluster
            next_index = (index + 1) % self.size
            while self.table[next_index] is not None:
                k, v = self.table[next_index]
                self.table[next_index] = None
                self.count -= 1
                self.insert(k, v)  # Reinsert
                next_index = (next_index + 1) % self.size
            
            return True
        return False
    
    def contains(self, key):
        """Kiểm tra key có tồn tại không"""
        return self.get(key) is not None
    
    def display(self):
        """Hiển thị toàn bộ hash table"""
        print(f"\n=== Hash Table (Linear Probing) [Count: {self.count}/{self.size}] ===")
        for i in range(self.size):
            if self.table[i] is not None:
                key, value = self.table[i]
                print(f"Index {i}: '{key}' -> {value}")
            else:
                print(f"Index {i}: None")
        print("=" * 50)

In [ ]:
# Kiểm tra HashTableWithLinearProbing
ht = HashTableWithLinearProbing(size=5)

# Thêm các phần tử
ht.insert("apple", 100)
ht.insert("banana", 200)
ht.insert("cherry", 300)
ht.insert(1, "one")
ht.insert(6, "six")  # 6 % 5 = 1 -> collision, sẽ tìm vị trí tiếp theo

ht.display()

# Truy xuất
print(f"\nget('apple'): {ht.get('apple')}")
print(f"get(1): {ht.get(1)}")
print(f"get(6): {ht.get(6)}")  # ✅ Tìm được!

# Xóa
ht.delete("banana")
ht.display()

## 8. So sánh các kỹ thuật xử lý Collision

### Chaining vs Linear Probing

| Đặc điểm | Chaining | Linear Probing |
|----------|----------|----------------|
| **Lưu trữ** | Mỗi bucket là list | Mỗi vị trí là một entry |
| **Khi collision** | Thêm vào list | Tìm vị trí trống tiếp theo |
| **Load factor** | Có thể > 1 | Phải ≤ 1 |
| **Xóa** | Đơn giản | Cần rehash |
| **Tốc độ** | Tốt khi ít collision | Nhanh hơn khi ít collision |

**Load Factor** = Số phần tử / Kích thước hash table
- Load factor càng cao → collision càng nhiều → hiệu suất giảm
- Thường giữ load factor < 0.7

## 9. Sử dụng Hash Table trong Python

### 9.1. Dictionary trong Python là Hash Table

Python's `dict` được triển khai bằng hash table với chaining!

In [ ]:
# Dictionary trong Python sử dụng hash table
my_dict = {
    "apple": 100,
    "banana": 200,
    "cherry": 300
}

# Các thao tác cơ bản
print("Thêm phần tử:")
my_dict["date"] = 400
print(my_dict)

print("\nTruy xuất:")
print(f"apple: {my_dict['apple']}")
print(f"banana: {my_dict.get('banana')}")
print(f"grape: {my_dict.get('grape', 'Not found')}")

print("\nKiểm tra tồn tại:")
print(f"'apple' in my_dict: {'apple' in my_dict}")

print("\nXóa phần tử:")
del my_dict["banana"]
print(my_dict)

# Hiệu suất
import time

# Tạo dữ liệu lớn
large_dict = {i: i*2 for i in range(100000)}
large_list = list(range(100000))

# So sánh tốc độ tìm kiếm
target = 99999

# Hash table (dict) - O(1)
start = time.time()
_ = large_dict[target]
dict_time = time.time() - start

# List - O(n)
start = time.time()
_ = large_list.index(target)
list_time = time.time() - start

print(f"\nSo sánh tốc độ tìm kiếm (phần tử {target}):")
print(f"Hash Table (dict): {dict_time*1000:.4f} ms")
print(f"List: {list_time*1000:.4f} ms")
print(f"Hash Table nhanh hơn {list_time/dict_time:.1f} lần!")

## 10. Ứng dụng thực tế của Hash Table

### 10.1. Đếm tần suất xuất hiện

In [ ]:
# Đếm tần suất xuất hiện của các từ trong văn bản
def count_frequency(text):
    """Đếm số lần xuất hiện của mỗi từ"""
    words = text.lower().split()
    frequency = {}  # Hash table
    
    for word in words:
        frequency[word] = frequency.get(word, 0) + 1
    
    return frequency

text = "the quick brown fox jumps over the lazy dog the fox is quick"
freq = count_frequency(text)

print("Tần suất xuất hiện của các từ:")
for word, count in sorted(freq.items()):
    print(f"  '{word}': {count}")

### 10.2. Kiểm tra phần tử trùng lặp

In [ ]:
def has_duplicate(arr):
    """Kiểm tra mảng có phần tử trùng lặp không (sử dụng hash set)"""
    seen = set()  # Hash set (chỉ lưu keys, không lưu values)
    
    for num in arr:
        if num in seen:
            return True
        seen.add(num)
    return False

# Kiểm tra
arr1 = [1, 2, 3, 4, 5]
arr2 = [1, 2, 3, 2, 4]

print(f"{arr1} có trùng lặp? {has_duplicate(arr1)}")
print(f"{arr2} có trùng lặp? {has_duplicate(arr2)}")

### 10.3. Tìm cặp phần tử có tổng bằng target (Two Sum)

In [ ]:
def two_sum(nums, target):
    """
    Tìm 2 số trong mảng có tổng bằng target
    Trả về indices của 2 số đó
    Sử dụng hash table để đạt O(n) thay vì O(n²)
    """
    num_map = {}  # {num: index}
    
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_map:
            return [num_map[complement], i]
        num_map[num] = i
    
    return None

# Kiểm tra
nums = [2, 7, 11, 15]
target = 9
result = two_sum(nums, target)
print(f"Input: nums = {nums}, target = {target}")
if result:
    print(f"Output: {result} (vì nums[{result[0]}] + nums[{result[1]}] = {nums[result[0]]} + {nums[result[1]]} = {target})")
else:
    print("Không tìm thấy!")

## 11. Tổng kết

### Ưu điểm của Hash Table:
- ✅ **Truy xuất cực nhanh**: O(1) trung bình
- ✅ **Thêm/xóa nhanh**: O(1) trung bình
- ✅ **Linh hoạt**: Hỗ trợ nhiều kiểu key
- ✅ **Thực tế**: Dictionary trong Python sử dụng hash table

### Nhược điểm:
- ⚠️ **Collision**: Cần xử lý khi nhiều key có cùng hash
- ⚠️ **Không có thứ tự**: Không giữ thứ tự thêm vào (Python 3.7+ giữ thứ tự)
- ⚠️ **Bộ nhớ**: Cần nhiều bộ nhớ hơn array đơn giản

### Khi nào sử dụng:
- ✅ Cần tìm kiếm/thêm/xóa nhanh (O(1))
- ✅ Cần lưu trữ cặp key-value
- ✅ Không cần duyệt theo thứ tự
- ✅ Dữ liệu không quá lớn (memory constraint)

### Time Complexity:

| Thao tác | Best Case | Average Case | Worst Case |
|----------|-----------|--------------|------------|
| **Search** | O(1) | O(1) | O(n) |
| **Insert** | O(1) | O(1) | O(n) |
| **Delete** | O(1) | O(1) | O(n) |

*Worst case xảy ra khi tất cả keys có cùng hash value (rất hiếm với hash function tốt)*

## 12. Bài tập thực hành

Hãy thử tự triển khai các chức năng sau:

1. **Triển khai Hash Table hoàn chỉnh** với load factor và resize
2. **Đếm số lần xuất hiện** của mỗi ký tự trong chuỗi
3. **Tìm phần tử đầu tiên không lặp lại** trong mảng
4. **Nhóm các từ đồng âm** (anagrams) sử dụng hash table

Bắt đầu thực hành ngay!